In [1]:
from signals import loadData, generate_signals, calculate_technical_indicators
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [2]:
# data = loadData()
data = pd.read_csv('EURUSD5.csv', delimiter='\t', header=None, names=['timeStamp', 'open', 'high', 'low', 'close', 'volume'])
data = data[['timeStamp', 'close']]
data.rename(columns={'close':'mid'}, inplace=True)
data = data.tail(287*30)
data

,timeStamp,mid
91390,2024-06-04 19:10,1.08800
91391,2024-06-04 19:15,1.08812
91392,2024-06-04 19:20,1.08818
91393,2024-06-04 19:25,1.08809
91394,2024-06-04 19:30,1.08805
...,...,...
99995,2024-07-16 16:35,1.08846
99996,2024-07-16 16:40,1.08837
99997,2024-07-16 16:45,1.08855
99998,2024-07-16 16:50,1.08874


In [3]:
richData = calculate_technical_indicators(data.tail(287))
richData

,timeStamp,mid,SMA_short,SMA_long,RSI,EMA_short,EMA_long,MACD,Signal_line
99713,2024-07-15 17:05,1.09035,NaN,NaN,NaN,NaN,NaN,NaN,NaN
99714,2024-07-15 17:10,1.09006,NaN,NaN,NaN,NaN,NaN,NaN,NaN
99715,2024-07-15 17:15,1.08997,NaN,NaN,NaN,NaN,NaN,NaN,NaN
99716,2024-07-15 17:20,1.09011,NaN,NaN,NaN,NaN,NaN,NaN,NaN
99717,2024-07-15 17:25,1.08980,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
99995,2024-07-16 16:35,1.08846,1.088370,1.088058,57.485030,1.088330,1.088275,0.000055,0.000050
99996,2024-07-16 16:40,1.08837,1.088360,1.088042,53.216374,1.088336,1.088282,0.000054,0.000051
99997,2024-07-16 16:45,1.08855,1.088371,1.088043,52.095808,1.088369,1.088302,0.000067,0.000054
99998,2024-07-16 16:50,1.08874,1.088399,1.088051,59.217877,1.088426,1.088334,0.000092,0.000062


In [4]:
is_signal_col = lambda name: 'mid' in name or 'quote' in name or 'SMA' in name or 'RSI' in name or 'MACD' in name or 'Signal' in name
fig = px.line(richData, x = 'timeStamp', y = [col for col in richData.columns if is_signal_col(col)])
fig.show()

In [5]:
# # signal = (data['SMA_10'] > data['SMA_50']).to_numpy() * 1
# nd = richData[['timeStamp', 'mid', 'MACD', 'Signal_line']]
# nd['signal'] = 0
# nd['macd_signal'] = None
# nd.loc[(nd['MACD'] > nd['Signal_line']), 'signal'] = 1
# nd.loc[(nd['MACD'] < nd['Signal_line']), 'signal'] = 0
# nd['signal'] = nd['signal'].diff()
# nd.loc[(nd['signal'] == -1), 'macd_signal'] = 0
# nd.loc[(nd['signal'] == 1), 'macd_signal'] = 1
# nd['macd_signal'] = nd['macd_signal'].astype('str')
# # nd['sma_signal'] = signal
# # nd.loc[(nd['sma_signal'] == 0)]
# nd.drop(columns=['signal'], inplace=True)

# nd

In [6]:
# is_signal_col = lambda name: 'mid' in name or 'quote' in name or 'SMA' in name or 'RSI' in name or 'MACD' in name or 'Signal' in name
# line = px.line(nd, x = 'timeStamp', y = [col for col in nd.columns if is_signal_col(col)])
# scatter = px.scatter(nd, x = 'timeStamp', y='mid', color='macd_signal', size='mid' )
# plot = go.Figure(data=line.data + scatter.data)
# plot.show()

In [8]:
import strategies
test_data = loadData(287, False)
sma = strategies.SMA(10, 50, is_statefull= False)
rsi = strategies.RSI(14, is_statefull= False)
macd = strategies.MACD(12, 26, 9, is_statefull= False)
composite = strategies.StateFullComposite([sma, rsi, macd])
actions = []
amounts = []
for i in range(1, len(test_data)+1):
    action, amount = composite.action( test_data.head(i) ) #macd.action( test_data.head(i) ) + rsi.action( test_data.head(i) ) + sma.action( test_data.head(i) )
    actions.append( action )
    amounts.append( amount )
test_data['signal'] = actions
test_data['amount'] = amounts

# test_data['signal'] = test_data['signal'].astype(str)
test_data['signal'].replace({0:'hold', 1: 'buy', -1:'sell', 2: 'strong_buy', 3:'very_strong_buy', -2: 'strong_sell', -3: 'very_strong_sell'}, inplace=True)
is_signal_col = lambda name: 'mid' in name or 'quote' in name or 'SMA' in name or 'RSI' in name or 'MACD' in name or 'Signal' in name
line = px.line(test_data, x = 'timeStamp', y = [col for col in test_data.columns if is_signal_col(col)])
scatter = px.scatter(test_data[~test_data['signal'].isin(['hold'])], x = 'timeStamp', y='mid', color='signal', size = 'mid' )
plot = go.Figure(data=line.data + scatter.data)
plot.show()

bought 354 units at 1.08949/unit| total holding: 2354 at avg buy price: 1.0650719881053528
bought 457 units at 1.08972/unit| total holding: 2811 at avg buy price: 1.0690791533262185
bought 404 units at 1.08974/unit| total holding: 3215 at avg buy price: 1.0716754152410577
bought 524 units at 1.09009/unit| total holding: 3739 at avg buy price: 1.074256116608719
bought 604 units at 1.09006/unit| total holding: 4343 at avg buy price: 1.0764540317752707
bought 418 units at 1.09007/unit| total holding: 4761 at avg buy price: 1.0776494685990339
sold 857 units at profit: 0.012440531400966126 avg buy price: 1.0776494685990339 current price: 1.09009
total profit: 528.3101627657986
bought 827 units at 1.0899/unit| total holding: 4731 at avg buy price: 1.0775373684210527
sold 880 units at profit: 0.012602631578947365 avg buy price: 1.0775373684210527 current price: 1.09014
total profit: 539.4004785552723
bought 480 units at 1.08992/unit| total holding: 4331 at avg buy price: 1.0763490394827986
bo